![Ejemplo 3](images/Ej3.png)

En este ejercicio utilizaremos la notación compacta general para cualquier número de intervalo y corrientes calientes y frías. Para ello, tendremos que leer de dos archivos Excel los aportes de las corrientes calientes y la eliminación de las frías. Estos datos, en forma de tabla, están en el archivo "Ejercicio3.xlsx", en las hojas "Hot" y "Cold", respectivamente.

En este caso, tendremos que usar la librería Pandas, con la que importaremos los datos para, posteriormente usarlos dentro del modelo de optimización.

In [ ]:
from pyomo.environ import *
import pandas as pd

model = ConcreteModel()

Leemos los datos de las corrientes calientes

In [ ]:
Hot_df = pd.read_excel('Datos/Ejercicio3.xlsx', 'Hot',skiprows = 1,index_col=0)
Hot_df

E igualmente de las frías.

In [ ]:
Cold_df = pd.read_excel('Datos/Ejercicio3.xlsx', 'Cold',skiprows = 1,index_col=0)
Cold_df

Las columnas de las tablas anteriores son los diferentes intervalos de temperatura (9+1 ya que incluimos un intervalo 0 que necesitamos porque en el balance aparece el término R(i-1)), mientras que las filas son las corrientes calientes y frías respectivamente. Con estos datos crearemos tres índices para el modelo: uno (h) contendrá los identificadores de las corrientes calientes (H1, H2 y H3), otro (c) de las frías (C1 y C2) y un último (i) con los diez intervalos (0-9)

In [ ]:
hotst = Hot_df.index.values.tolist()
coldst = Cold_df.index.values.tolist()
model.h=Set(initialize=hotst)
model.c=Set(initialize=coldst)
model.i=Set(initialize=Hot_df.columns)

Con int_max calculamos el número total de intervalos

In [ ]:
int_max=max(Hot_df.columns.values.tolist())

Podemos ver lo que hemos creado hasta ahora haciendo un model.pprint()

In [ ]:
model.pprint()

Creamos ahora las variables del modelo. Hay i potenciales aportes de calor por utilities calientes (Qs), uno en cada intervalo, al igual que de frías (Qw). De esta forma damos la oportunidad de que entren las utilities en cualquier nivel de temperatura. Creamos también i residuos (R), aunque sabemos que no habrá residuo del intervalo 0 ni tampoco del intervalo 9, lo cual implementaremos como dos restricciones diferentes.

In [ ]:
Qs = model.Qs = Var(model.i,within = NonNegativeReals)
Qw = model.Qc = Var(model.i,within = NonNegativeReals)
R = model.R = Var(model.i,within = NonNegativeReals)

Buscamos la minimización del vapor del intervalo 1 más la utility fría del intervalo 9.

In [ ]:
model.util = Objective(expr = model.Qs[1] + model.Qc[int_max])

Constraints

In [ ]:
ni = list(model.i)[1:] #Creamos una lista que contenga los índices sobre los que haremos el balance. En nuestro caso el balance lo aplicamos en todos los intervalos, excepto el intervalo 0. Es decir sobre 1-9, lo cual expresamos por [1:], es decir, desde 1 hasta el final de la lista.
model.int = ConstraintList()
for i in ni:
    model.int.add(
        R[i-1]+Qs[i]+sum(Hot_df[i]) == R[i]+sum(Cold_df[i])+Qw[i]
    )

model.R0 = Constraint(expr = R[0] == 0) #No hay residuo ni desde intervalo 0 ni desde 9
model.R9 = Constraint(expr = R[int_max] == 0)

nii = list(model.i)[0:1]+list(model.i)[2:] #Con esta restricción buscamos que solo se aporte utility en el primer intervalo (no más abajo), por lo que decimos que desde 2 a 9 la variable Qs sea 0 y también que Qs[0] sea 0. Es decir, que solo dejamos como variable Qs[1]
model.steam = ConstraintList()
for i in nii:
    model.steam.add(
        Qs[i]==0
    )

niii = list(model.i)[0:int_max] #Lo mismo deantes para la utility fría. Es decir de 0 a 8 (el 9 no está incluido en el intervalo al expresarlo así en Python) Qw es 0.
model.cw = ConstraintList()
for i in niii:
    model.cw.add(
        Qw[i]==0
    )

Resolvemos el modelo

In [ ]:
results = SolverFactory('glpk').solve(model)
model.pprint()
results.write()

In [ ]:
Qh = value(model.Qs[1])
Qc = value(model.Qc[int_max])
print('Cold utility = {0:2.1f}, Hot utility = {1:2.1f}'.format(Qc, Qh))